In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-21 17:53:41--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv.11’

customer_churn.csv. 100%[===================>] 668.81K  --.-KB/s    in 0.006s  

2019-04-21 17:53:42 (106 MB/s) - ‘customer_churn.csv.11’ saved [684858/684858]



In [0]:
from fastai.tabular import * 
import pandas as pd

In [0]:
df=pd.read_csv('customer_churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
procs = [FillMissing, Categorify, Normalize]

In [0]:
valid_idx = range(len(df)-2000, len(df))

In [0]:
dep_var = 'Exited'
cat_names = ['Geography','Gender','HasCrCard','IsActiveMember']
cont_names = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

In [0]:
data = TabularDataBunch.from_df('',df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names,cont_names=cont_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']


In [0]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[1 2 2 2]
 [1 1 2 2]
 [2 2 1 2]
 [1 2 2 1]
 [3 2 2 2]]
[[-0.704013 -0.850317 -1.044506 -1.230944  0.809463  0.553358]
 [-1.468126  2.38432  -1.044506 -1.230944 -0.905501 -1.636221]
 [-0.177394  0.29132  -0.697811  0.78923  -0.905501 -1.536779]
 [ 0.638348 -0.564908 -1.044506  1.700369 -0.905501 -1.688644]
 [-0.435541 -1.135726 -0.351116  1.226536 -0.905501  1.734141]]
[0 0 0 0 0]


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.395767,0.346035,0.859000,00:01


In [0]:
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.6583, 0.3417]))

Run it five times to match to the number of runs in tensor flow. 

In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.432355,0.402169,0.830000,00:01
1,0.362049,0.367887,0.853500,00:01
2,0.351247,0.355200,0.855500,00:01
3,0.348764,0.344052,0.859000,00:01
4,0.332360,0.345239,0.861500,00:01


In [0]:
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.5812, 0.4188]))